In [4]:
import os, json, uuid
from datetime import datetime, timezone, timedelta
from pymongo import MongoClient, ReturnDocument
from pymongo.errors import DuplicateKeyError
from dotenv import load_dotenv
from bson.decimal128 import Decimal128

# ---------------------------
# Config & Connect
# ---------------------------
load_dotenv()
uri = os.getenv("MONGODB_URI")
database_name = os.getenv("MONGODB_DATABASE")
collection_name = "sales_invoices"

if not uri or not database_name:
    raise RuntimeError("Missing MONGODB_URI or MONGODB_DATABASE.")

client = MongoClient(uri)
db = client[database_name]
col = db[collection_name]

# ---------------------------
# Input payload to PATCH into the only empty form
# (Replace with your runtime payload)
# ---------------------------
payload = {
    "id": "2b877c11-acbc-40f4-80c5-9bd13bb2d71f",
    "current_form_qr_code": "20251013",
    "previous_form_qr_code": "20251012",
    "tin": "123-456-789",
    "location": "JEF Gas Station – Sikatuna Branch",
    "created": "2025-10-13T05:03:46.557977Z",
    "date": "2025-10-13",
    "items": [
        {"receipt_number": "123","customer_name": "Markus","type": "fuel","vatable_sales": 1111,"vat_amount": 133.32,"total_amount": 1244.32},
        {"receipt_number": "222","customer_name": "Just","type": "fuel","vatable_sales": 22222,"vat_amount": 2666.64,"total_amount": 24888.64}
    ],
    "cashier_employee_number": "10001",
    "cashier": "John Smith",
    "recorder_employee_number": "10003",
    "recorder": "David Williams"
}

# ---------------------------
# Helpers
# ---------------------------
REQUIRED_TOP_LEVEL = [
    "id","current_form_qr_code","previous_form_qr_code","tin","location","date","items",
    "cashier_employee_number","cashier","recorder_employee_number","recorder"
]

def _to_decimal128(x):
    return x if isinstance(x, Decimal128) else Decimal128(str(x))

def _normalize_payload(doc):
    d = dict(doc)
    items = []
    for it in d.get("items", []):
        it = dict(it)
        for k in ("vatable_sales","vat_amount","total_amount"):
            if k in it and it[k] is not None:
                it[k] = _to_decimal128(it[k])
        items.append(it)
    d["items"] = items
    return d

def _validate_payload(d):
    missing = [k for k in REQUIRED_TOP_LEVEL if k not in d]
    if missing:
        raise RuntimeError(f"❌ Payload missing required fields: {missing}")
    if not isinstance(d.get("items"), list) or len(d["items"]) == 0:
        raise RuntimeError("❌ Payload.items must be a non-empty list.")

def _ensure_single_empty_index_and_sync():
    try:
        col.create_index(
            [("is_empty", 1)],
            unique=True,
            name="uniq_is_empty_true",
            partialFilterExpression={"is_empty": True},
            background=True,
        )
    except Exception:
        pass
    col.update_many({"items": {"$exists": False}}, {"$set": {"items": []}})
    col.update_many({"items": {"$exists": True, "$size": 0}}, {"$set": {"is_empty": True}})
    col.update_many({"items": {"$exists": True, "$ne": []}}, {"$set": {"is_empty": False}})

def _patch_only_empty(norm_payload):
    count_empty = col.count_documents({"is_empty": True})
    if count_empty == 0:
        raise RuntimeError("❌ No document with is_empty=True found. Nothing to patch.")
    if count_empty > 1:
        raise RuntimeError(f"❌ Expected exactly one is_empty=True document, found {count_empty}.")
    only_empty = col.find_one({"is_empty": True})
    if not only_empty:
        raise RuntimeError("❌ Race condition: is_empty=True doc disappeared.")
    update_op = {
        "$set": {
            "id": norm_payload["id"],
            "current_form_qr_code": norm_payload["current_form_qr_code"],
            "previous_form_qr_code": norm_payload["previous_form_qr_code"],
            "tin": norm_payload["tin"],
            "location": norm_payload["location"],
            "date": norm_payload["date"],
            "cashier_employee_number": norm_payload["cashier_employee_number"],
            "cashier": norm_payload["cashier"],
            "recorder_employee_number": norm_payload["recorder_employee_number"],
            "recorder": norm_payload["recorder"],
            "items": norm_payload["items"],
            "updated_at": datetime.now(timezone.utc).isoformat(),
            "is_empty": False
        }
    }
    patched = col.find_one_and_update(
        {"_id": only_empty["_id"], "is_empty": True},
        update_op,
        return_document=ReturnDocument.AFTER
    )
    if patched is None:
        raise RuntimeError("❌ Patch failed — the target doc was modified by someone else.")
    return patched

def _create_new_empty_after(patched_doc):
    # Build new empty form using patched doc as prior context
    now_utc = datetime.now(timezone.utc)
    prev_date = str(patched_doc.get("date") or datetime.now().date().isoformat())
    try:
        prev_date_dt = datetime.fromisoformat(prev_date)
    except ValueError:
        prev_date_dt = datetime.fromisoformat(prev_date + "T00:00:00")
    new_date = (prev_date_dt + timedelta(days=1)).date().isoformat()

    new_doc = {
        "id": str(uuid.uuid4()),
        "previous_form_qr_code": patched_doc.get("current_form_qr_code",""),
        "current_form_qr_code": now_utc.strftime("%Y%m%d"),
        "tin": patched_doc.get("tin",""),
        "location": patched_doc.get("location",""),
        "created": now_utc.isoformat(),
        "date": new_date,
        "is_empty": True,
    }

    try:
        res = col.insert_one(new_doc)
        new_doc["_id"] = str(res.inserted_id)
        return new_doc
    except DuplicateKeyError:
        existing_empty = col.find_one({"is_empty": True}, sort=[("created", -1)])
        return existing_empty

# ---------------------------
# Orchestrate: ensure index/sync → PATCH the only empty → CREATE a fresh empty
# ---------------------------
_ensure_single_empty_index_and_sync()
_validate_payload(payload)
norm = _normalize_payload(payload)

patched = _patch_only_empty(norm)
print("✅ Patched the only is_empty=True document and set is_empty=False.")
print(json.dumps(patched, indent=2, default=str))

# After patch, there should be no is_empty=True; create a fresh empty for the next run
fresh_empty = _create_new_empty_after(patched)
print("✅ Created (or reused concurrent) new EMPTY form.")
print(json.dumps(fresh_empty, indent=2, default=str))


✅ Patched the only is_empty=True document and set is_empty=False.
{
  "_id": "68ed2e0785573c9c4a8ff7b6",
  "id": "2b877c11-acbc-40f4-80c5-9bd13bb2d71f",
  "previous_form_qr_code": "20251012",
  "current_form_qr_code": "20251013",
  "tin": "123-456-789",
  "location": "JEF Gas Station \u2013 Sikatuna Branch",
  "created": "2025-10-13T16:51:19.899417+00:00",
  "date": "2025-10-13",
  "items": [
    {
      "receipt_number": "123",
      "customer_name": "Markus",
      "type": "fuel",
      "vatable_sales": "1111",
      "vat_amount": "133.32",
      "total_amount": "1244.32"
    },
    {
      "receipt_number": "222",
      "customer_name": "Just",
      "type": "fuel",
      "vatable_sales": "22222",
      "vat_amount": "2666.64",
      "total_amount": "24888.64"
    }
  ],
  "is_empty": false,
  "cashier": "John Smith",
  "cashier_employee_number": "10001",
  "recorder": "David Williams",
  "recorder_employee_number": "10003",
  "updated_at": "2025-10-13T16:52:33.238824+00:00"
}
✅ Cre